**Vérification des attributs des couches contenues dans le GPKG POINT = Cotation** (py8) ArcGIS Pro

Ce programme vérifie l'ensemble des couches du GPKG pour savoir si les attributs PROF et Zreseau sont présents (donc positifs), puis les extrait dans des shapefiles individuels par couche.

![image_py8](../images_colab/py8.png)

In [ ]:
import arcpy
import os

gpkg_path = r"C:\Users\liege\Desktop\TER_SHAPES\POINT\POINT.gpkg" #gpkg POINT
dossier_sortie = r"C:\Users\liege\Downloads\POINT_ATTRIBUT"
nom_champ_prof = "PROF"
nom_champ_zreseau = "Zreseau"

arcpy.env.workspace = gpkg_path
couches = arcpy.ListFeatureClasses()

for couche in couches:
    champs = [f.name for f in arcpy.ListFields(couche)]
    if nom_champ_prof not in champs or nom_champ_zreseau not in champs:
        print(f"La couche {couche} ne contient pas les champs {nom_champ_prof} et {nom_champ_zreseau}.")
        continue

    # Expression pour sélectionner les entités avec PROF < 0 ou Zreseau < 0
    expression = f'"{nom_champ_prof}" < 0 OR "{nom_champ_zreseau}" < 0'

    # Création d’une couche temporaire avec cette sélection
    couche_temp = "temp_layer"
    arcpy.MakeFeatureLayer_management(couche, couche_temp, expression)

    # Vérifier si la couche temporaire contient des entités
    count = int(arcpy.GetCount_management(couche_temp).getOutput(0))
    if count == 0:
        print(f" Aucune erreur d'attribut dans la couche {couche}")
        arcpy.Delete_management(couche_temp)
        continue

    # Construire le chemin de sortie avec mention "erreurs_ATTRIBUTS"
    nom_sortie = f"erreurs_ATTRIBUTS_{couche}.shp"
    sortie_path = os.path.join(dossier_sortie, nom_sortie)

    # Exporter les entités sélectionnées
    arcpy.CopyFeatures_management(couche_temp, sortie_path)
    arcpy.Delete_management(couche_temp)

    print(f"Exporté les entités avec erreurs d'attributs dans : {sortie_path}")


**Vérification des attributs IDRegard des couches contenues dans le GPKG SYMBOLE, pour les réseaux EP et EU (les seuls avec des ID)** (py9) ArcGIS Pro

Ce programme vérifie les 2 couches du GPKG afin de détecter si un attribut IDRegard a été attribué à deux entités différentes, puis extrait ces entités dans des shapefiles individuels par couche. (Pour certains symboles, c’est normal.)

![image_py9](../images_colab/py9.png)

In [ ]:
import arcpy
import os
from collections import Counter

gpkg_path = r"C:\Users\liege\Desktop\TER_SHAPES\SYMBOLE\SYMBOLE.gpkg" #gpkg SYMBOLE
dossier_sortie = r"C:\Users\liege\Downloads\SYMBOLE_ATTRIBUT"
champ_id = "IDRegard"

couches = [
    "Symboles_EP_EtiArcGIS",
    "Symboles_EU_EtiArcGIS"
]

arcpy.env.workspace = gpkg_path
os.makedirs(dossier_sortie, exist_ok=True)

def clean_export_shapefile(input_layer, output_path):
    """Exporte en shapefile en supprimant les champs non compatibles"""
    if not arcpy.Exists(input_layer):
        raise RuntimeError(f"La couche {input_layer} n'existe pas.")

    fields = arcpy.ListFields(input_layer)
    shp_types = ['Integer', 'SmallInteger', 'Double', 'Single', 'String', 'Date']

    # Création d'un FieldMappings pour ne garder que les champs compatibles
    field_mappings = arcpy.FieldMappings()
    for field in fields:
        if field.type in shp_types and len(field.name) <= 10:
            fm = arcpy.FieldMap()
            fm.addInputField(input_layer, field.name)
            field_mappings.addFieldMap(fm)

    # Export propre
    arcpy.FeatureClassToFeatureClass_conversion(
        input_layer,
        os.path.dirname(output_path),
        os.path.basename(output_path),
        field_mapping=field_mappings
    )

for couche in couches:
    print(f"\n Traitement de la couche : {couche}")

    if not arcpy.Exists(couche):
        print(f"La couche {couche} n'existe pas dans le gpkg.")
        continue

    champs = [f.name for f in arcpy.ListFields(couche)]
    if champ_id not in champs:
        print(f"Le champ '{champ_id}' est absent dans {couche}.")
        continue

    # Récupérer tous les ID non nuls
    ids = []
    with arcpy.da.SearchCursor(couche, [champ_id]) as cursor:
        for row in cursor:
            if row[0] is not None:
                ids.append(row[0])

    # Chercher les ID en doublon
    compteur = Counter(ids)
    ids_dupliques = [id_val for id_val, count in compteur.items() if count > 1]

    if not ids_dupliques:
        print(f"Pas d'ID doublon dans la couche {couche}.")
        continue

    # Construire expression SQL adaptée (en fonction du type champ)
    field_obj = arcpy.ListFields(couche, champ_id)[0]
    if field_obj.type in ['String', 'Guid']:
        expression = " OR ".join([f"{champ_id} = '{val}'" for val in ids_dupliques])
    else:
        expression = " OR ".join([f"{champ_id} = {val}" for val in ids_dupliques])

    temp_layer = "temp_dups"
    # Supprimer si existe déjà
    if arcpy.Exists(temp_layer):
        arcpy.Delete_management(temp_layer)

    arcpy.MakeFeatureLayer_management(couche, temp_layer, where_clause=expression)

    count = int(arcpy.GetCount_management(temp_layer).getOutput(0))
    if count == 0:
        print(f"Aucun doublon trouvé après filtrage pour {couche}.")
        arcpy.Delete_management(temp_layer)
        continue

    sortie_path = os.path.join(dossier_sortie, f"duplicates_{couche}.shp")

    # Exporter en shapefile nettoyé des champs incompatibles
    clean_export_shapefile(temp_layer, sortie_path)

    arcpy.Delete_management(temp_layer)
    print(f"Exporté les entités avec ID doublons dans : {sortie_path}")



**Création d’un gpkg avec une seule couche contenant les buffers selon les classes de réseaux** (py10) COLAB

Ce programme va lire le géopackage des tronçons et va fusionner l’ensemble des entités dans un seul fichier si elles contiennent une classe. Un champ avec le nom des couches sera créé. Trois types de buffer possibles : classe A = 0,5 m, classe B = 1,5 m et classe C = 3 m.

![image_py10](../images_colab/py10.png)

In [ ]:
import os
import geopandas as gpd
import pandas as pd
import fiona
from shapely.geometry import Polygon
from pyproj import CRS

gpkg_path = "/content/LINEAIRE.gpkg"  #gpkg LINEAIRE

output_folder = "buffers_output"
os.makedirs(output_folder, exist_ok=True)
output_gpkg = os.path.join(output_folder, "buffers_classes.gpkg")

def is_metric_crs(crs):
    try:
        crs_obj = CRS(crs)
        unit = crs_obj.axis_info[0].unit_name.lower()
        return "metre" in unit or "meter" in unit
    except:
        return False

buffer_sizes = {
    "A": 0.5,
    "B": 1.5,
    "C": 3.0
}

layers = fiona.listlayers(gpkg_path)
print(f"Couches détectées : {layers}")

buffer_list = []

for layer_name in layers:
    print(f"Traitement de la couche : {layer_name}")
    gdf = gpd.read_file(gpkg_path, layer=layer_name)

    if 'classe' not in gdf.columns and 'Classe' in gdf.columns:
        gdf = gdf.rename(columns={'Classe': 'classe'})
    if 'classe' not in gdf.columns:
        print(f" - Pas de champ 'classe' dans {layer_name}, couche ignorée.")
        continue

    gdf_sel = gdf[gdf['classe'].isin(buffer_sizes.keys())].copy()
    if gdf_sel.empty:
        print(f" - Pas d'entités des classes A, B, C dans {layer_name}")
        continue

    gdf_sel['layer'] = layer_name

    if not is_metric_crs(gdf_sel.crs):
        print("  CRS non métrique détecté, reprojection en UTM...")
        gdf_sel = gdf_sel.to_crs(gdf_sel.estimate_utm_crs())

    def buffer_by_class(row):
        dist = buffer_sizes.get(row['classe'], 0)
        return row.geometry.buffer(dist)

    gdf_sel['geometry'] = gdf_sel.apply(buffer_by_class, axis=1)
    gdf_sel['buffer_size'] = gdf_sel['classe'].map(buffer_sizes)

    keep_fields = ['classe', 'layer', 'buffer_size', 'geometry']
    drop_cols = [col for col in gdf_sel.columns if col not in keep_fields]
    gdf_sel = gdf_sel.drop(columns=drop_cols)

    buffer_list.append(gdf_sel)

if buffer_list:
    final_gdf = gpd.GeoDataFrame(pd.concat(buffer_list, ignore_index=True), crs=buffer_list[0].crs)
    final_gdf.to_file(output_gpkg, layer='buffers_classes', driver="GPKG")
    print(f"Buffers sauvegardés dans {output_gpkg}")
else:
    print("Aucune entité à bufferiser.")


**Création d’un GPKG avec une seule couche contenant les tronçons trop proches d’autres tronçons de type différent (< 20 cm)** (py11) COLAB

Ce programme va lire le géopackage des tronçons et va sélectionner puis fusionner dans une même couche les tronçons de types différents proches de moins de 20 cm.

![image_py11](../images_colab/py11.png)

In [ ]:
import geopandas as gpd
import pandas as pd
import fiona

gpkg_path = "/content/LINEAIRE.gpkg"  # gpkg LINEAIRE

layers = fiona.listlayers(gpkg_path)
print(f"Couches détectées : {layers}")

gdfs = []
for layer in layers:
    gdf = gpd.read_file(gpkg_path, layer=layer)
    # Vérifier que c'est un GeoDataFrame avec géométrie et que ce n'est pas vide
    if hasattr(gdf, "geometry") and not gdf.empty:
        # Vérifier que la première géométrie existe et est un LineString/MultiLineString
        if gdf.geometry.iloc[0] is not None and gdf.geometry.iloc[0].geom_type in ['LineString', 'MultiLineString']:
            gdf['source_layer'] = layer
            gdfs.append(gdf)

if not gdfs:
    raise RuntimeError("Aucune couche linéaire trouvée dans ce GeoPackage.")

merged = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True), crs=gdfs[0].crs)
print(f"Total entités fusionnées : {len(merged)}")

merged['buffer'] = merged.geometry.buffer(0.2)

sindex = merged.sindex

close_indices = set()
for idx, buffered_geom in merged['buffer'].items():
    possible_matches = list(sindex.intersection(buffered_geom.bounds))
    if idx in possible_matches:
        possible_matches.remove(idx)

    current_layer = merged.at[idx, 'source_layer']

    for other_idx in possible_matches:
        if merged.at[other_idx, 'source_layer'] != current_layer:
            if buffered_geom.intersects(merged.at[other_idx, 'buffer']):
                close_indices.add(idx)
                close_indices.add(other_idx)

print(f"Nombre d'entités trop proches (<20cm) entre différentes couches : {len(close_indices)}")

proches = merged.loc[list(close_indices)].copy()
proches.drop(columns=['buffer'], inplace=True)

output_path = "entites_trop_proches.gpkg"
proches.to_file(output_path, layer="entites_proches", driver="GPKG")

print(f"Export terminé vers : {output_path}")


**Vérification de la profondeur des réseaux d’un point de vue légal (information de cotation)** (py12) COLAB

Ce programme va lire les géopackages qui permettent de coter les tronçons et va comparer le champ PROF ou ZTampon - Zradier aux valeurs légales de profondeur actuelles (en ne gardant que les entités ayant les champs requis complets).

![image_py12](../images_colab/py12.png)

In [ ]:
import geopandas as gpd
import fiona
import os

# Dictionnaire des profondeurs légales par type réseau (majuscule)
profondeur_legale = {
    "AEP": 1.0,
    "EP": 1.0,
    "EU": 1.0,
    "SLT": 0.6,
    "HTA": 0.6,
    "ECL": 0.6,
    "BT": 0.6,
    "HYD": 0.7
}

point_gpkg = "/content/POINT.gpkg" # gpkg POINT
symbole_gpkg = "/content/SYMBOLE.gpkg" # gpkg SYMBOLE

def get_existing_field(fields, candidates):
    """Retourne le premier champ existant dans la liste candidates, ou None."""
    for c in candidates:
        if c in fields:
            return c
    return None

def detect_non_conformes(gdf, profondeur_fields, type_field, est_symbole=False):
    """Filtre les entités avec profondeur non conforme selon la réglementation."""
    non_conformes = []
    for idx, row in gdf.iterrows():
        type_reseau = row.get(type_field)
        if not isinstance(type_reseau, str):
            continue
        type_reseau = type_reseau.upper()

        if type_reseau not in profondeur_legale:
            # Type non reconnu, on ignore l’entité
            continue
        seuil = profondeur_legale[type_reseau]

        if est_symbole:
            ztampon = row.get(profondeur_fields[0], None)
            zradier = row.get(profondeur_fields[1], None)
            if ztampon in [None, 0] or zradier in [None, 0]:
                continue
            try:
                prof = float(ztampon) - float(zradier)
            except:
                continue
            if prof <= 0:
                continue
        else:
            prof = row.get(profondeur_fields[0], None)
            if prof is None:
                continue
            try:
                prof = float(prof)
            except:
                continue
            if prof < 0:
                continue

        if prof < seuil:
            non_conformes.append(row)

    if non_conformes:
        return gpd.GeoDataFrame(non_conformes, columns=gdf.columns)
    else:
        return None

rapport = []

# --- Analyse POINT.gpkg ---
print("Analyse du GeoPackage POINT...")
point_layers = fiona.listlayers(point_gpkg)

for layer in point_layers:
    print(f"Traitement couche POINT : {layer}")
    gdf = gpd.read_file(point_gpkg, layer=layer)

    champ_prof = get_existing_field(gdf.columns, ["PROF", "Prof", "prof"])
    champ_type = get_existing_field(gdf.columns, ["TYPE", "Type", "type"])
    if champ_prof is None or champ_type is None:
        print(f"Champs PROF ou TYPE absents dans la couche {layer}, passage.")
        continue

    result = detect_non_conformes(gdf, [champ_prof], champ_type, est_symbole=False)
    if result is not None:
        result['source_layer'] = layer
        rapport.append(result)

# --- Analyse SYMBOLE.gpkg ---
print("\nAnalyse du GeoPackage SYMBOLE...")
symbole_layers = ["Symboles_EU_EtiArcGIS", "Symboles_EP_EtiArcGIS"]

for layer in symbole_layers:
    print(f"Traitement couche SYMBOLE : {layer}")
    gdf = gpd.read_file(symbole_gpkg, layer=layer)

    champ_type = get_existing_field(gdf.columns, ["TYPE", "Type", "type"])
    champ_ztampon = get_existing_field(gdf.columns, ["ZTampon", "Ztampon", "ztampon"])
    champ_zradier = get_existing_field(gdf.columns, ["ZRadier", "Zradier", "zradier"])

    if champ_type is None or champ_ztampon is None or champ_zradier is None:
        print(f"Champs TYPE, ZTampon ou ZRadier absents dans la couche {layer}, passage.")
        continue

    result = detect_non_conformes(gdf, [champ_ztampon, champ_zradier], champ_type, est_symbole=True)
    if result is not None:
        result['source_layer'] = layer
        rapport.append(result)

# --- Export shapefile dans Colab (répertoire courant) ---
if rapport:
    import pandas as pd
    rapport_total = pd.concat(rapport, ignore_index=True)

    sortie = "rapport_profondeur_non_conforme.shp"
    rapport_total.to_file(sortie, driver="ESRI Shapefile")
    print(f"\nRapport généré : {sortie}")
else:
    print("\nAucune entité non conforme détectée.")
